# Image classification: cats & dogs

In [ ]:
# Handle imports up-front
import glob
import random
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing import image


## 1. Data preparation

### 1.1. Load the data paths

In [ ]:
# Set the path to the training data
training_data_path='../data/train'

# Get a list of training dog and cat images
training_dogs=glob.glob(f'{training_data_path}/dog/dog.*')
training_cats=glob.glob(f'{training_data_path}/cat/cat.*')

### 1.2. Inspect

In [ ]:
fig, axs = plt.subplots(3,2,figsize=(6, 4))

for cat, dog, row in zip(training_cats, training_dogs, axs):
    for animal, ax in zip([cat, dog], row):
        animal=image.load_img(animal)
        animal=image.img_to_array(animal)
        animal/=255.0
        ax.imshow(animal)
        ax.axis('off')

plt.tight_layout()
plt.show()

## 2. EDA

Let's take a look at a few of our images to get a feel for how image data is structured.

### 2.1. Image data

In [ ]:
# Load one of the dogs
dog=image.load_img(training_dogs[0])

# And convert it to an array - this is how TensorFlow will handel the data
dog=image.img_to_array(dog)

# Take a look at some properites of the object
print(f'Image data is: {type(dog)}')
print(f'Image data shape: {dog.shape}')

The data has shape of 500 x 390 x 3? So, the image is 500 x 390 pixles, but what is the 3? Let's plot the pixle values and you will see what is going on:

In [ ]:
plt.hist(dog[:,:,0].flatten(), bins=100, color='red', alpha=0.5, label='Red channel')
plt.hist(dog[:,:,1].flatten(), bins=100, color='green', alpha=0.5, label='Green channel')
plt.hist(dog[:,:,2].flatten(), bins=100, color='blue', alpha=0.5, label='Blue channel')
plt.xlabel('Pixel value')
plt.ylabel('Count')
plt.legend(loc='best')
plt.show()

There are a few intresting observations we can make here:

1. This array has 3 axies: 500 x 390 x 3. The first two are the dimensions of the image, the third is the three color channels: red, green and blue
2. 500 x 390 x 3 is over a half million individual values - this one image is 10 time more data that any of the other datasets we have worked with so far!
3. The range of pixle values is from 0 to about 250 - in reality it is (0,255) for a total range of 256 values per pixle. This is defined by the JPEG image standard.

### 2.2. Image dimensions

Let's take a look at a random sample of images from the dataset and see what their dimensions are.

In [ ]:
sample_size=1000
sample=random.sample(training_dogs, sample_size//2)
sample+=random.sample(training_cats, sample_size//2)

heights=[]
widths=[]

for sample_image in sample:

    sample_image=image.load_img(sample_image)
    sample_image=image.img_to_array(sample_image)
    heights.append(sample_image.shape[0])
    widths.append(sample_image.shape[1])

plt.hist(heights, bins=50, alpha=0.5, label='Image heights')
plt.hist(widths, bins=50, alpha=0.5, label='Image widths')
plt.xlabel('Image dimension')
plt.ylabel('Count')
plt.legend(loc='best')
plt.show()

Based on the above plot, let's set our image dimension at 128x128.

In [ ]:
image_dim=128

### 2.3. Image aspect ratios

In [ ]:
aspect_ratios=np.array(widths)/np.array(heights)

plt.hist(aspect_ratios, bins=50, color='black')
plt.xlabel('Image aspect ratio')
plt.ylabel('Count')
plt.show()

## 2. Model

### 2.1. Prepare images for streaming

In [ ]:
def make_datasets(training_data_path: str, image_dim: int, batch_size: int=32):

    training_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='training',
        seed=315,
        image_size=(image_dim, image_dim),
        color_mode='grayscale',
        batch_size=batch_size
    ).repeat()

    validation_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='validation',
        seed=315,
        image_size=(image_dim, image_dim),
        color_mode='grayscale',
        batch_size=batch_size
    ).repeat()

    # AUTOTUNE=tf.data.AUTOTUNE

    # training_dataset=training_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    # validation_dataset=validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

    return training_dataset, validation_dataset

training_dataset, validation_dataset=make_datasets(training_data_path, image_dim)

### 2.1. Model definition

In [ ]:
def compile_model(image_dim, learning_rate):

    initializer=tf.keras.initializers.GlorotUniform(seed=315)

    model=Sequential([
        layers.Input((image_dim, image_dim, 1)),
        layers.Rescaling(1./255),
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_initializer=initializer),
        layers.Dense(1, activation='sigmoid', kernel_initializer=initializer)
    ])

    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

    return model

model=compile_model(image_dim, 0.001)
model.summary()

### 2.2. Model training

In [ ]:
training_results=model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=5,
  steps_per_epoch=10,
  validation_steps=5
)

In [ ]:
# Set-up a 1x2 figure for accuracy and binary cross-entropy
fig, axs=plt.subplots(1,2, figsize=(8,4))

# Add the main title
fig.suptitle('CNN training curves')

# Plot training and validation accuracy
axs[0].set_title('Accuracy')
axs[0].plot(np.array(training_results.history['binary_accuracy']) * 100, label='Training')
axs[0].plot(np.array(training_results.history['val_binary_accuracy']) * 100, label='Validation')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy (%)')
axs[0].legend(loc='upper left')

# Plot training and validation binary cross-entropy
axs[1].set_title('Binary cross-entropy')
axs[1].plot(training_results.history['loss'])
axs[1].plot(training_results.history['val_loss'])
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Binary cross-entropy')

# Show the plot
plt.tight_layout()
plt.show()

### 2.3. Model optimization

In [ ]:
learning_rates=[0.001,0.0001,0.00001]
batch_sizes=[32,64,128,256]
training_results=[]

conditions=list(itertools.product(batch_sizes, learning_rates))

for i, parameters in enumerate(conditions):

    batch_size, learning_rate=parameters

    print(f'\nStarting training run {i + 1} of {len(learning_rates) * len(batch_sizes)}: batch size: {batch_size}, learning rate: {learning_rate}')

    training_dataset, validation_dataset=make_datasets(training_data_path, image_dim, batch_size)
    model=compile_model(image_dim, learning_rate)

    early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

    training_result=model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=10,
        steps_per_epoch=10,
        validation_steps=5,
        callbacks=[early_stopping]
    )

    training_results.append(training_result)

In [ ]:
# Set-up a 1x2 figure for accuracy and binary cross-entropy
fig, axs=plt.subplots(len(training_results), 2, figsize=(8,3*len(training_results)))

# Add the main title
fig.suptitle('CNN training curves', size='large')

# Plot the results of each training run
for i, parameters in enumerate(conditions):
    batch_size, learning_rate=parameters
    training_result=training_results[i]

    # Plot training and validation accuracy
    axs[i,0].set_title(f'Accuracy\nbatch size = {batch_size}, learning rate = {learning_rate}')
    axs[i,0].plot(np.array(training_result.history['binary_accuracy']) * 100, label='Training')
    axs[i,0].plot(np.array(training_result.history['val_binary_accuracy']) * 100, label='Validation')
    axs[i,0].set_xlabel('Epoch')
    axs[i,0].set_ylabel('Accuracy (%)')
    axs[i,0].legend(loc='best')

    # Plot training and validation binary cross-entropy
    axs[i,1].set_title(f'Binary cross-entropy\nbatch size = {batch_size}, learning rate = {learning_rate}')
    axs[i,1].plot(training_result.history['loss'], label='Training')
    axs[i,1].plot(training_result.history['val_loss'], label='Validation')
    axs[i,1].set_xlabel('Epoch')
    axs[i,1].set_ylabel('Binary cross-entropy')
    axs[i,1].legend(loc='best')

# Show the plot
plt.tight_layout()
plt.show()